In [6]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf


In [9]:
df=pd.read_csv('apple.csv')
df.head()

,Date,Close,Volume,Open,High,Low
0,02/28/2020,$273.36,106721200,$257.26,$278.41,$256.37
1,02/27/2020,$273.52,80151380,$281.1,$286,$272.96
2,02/26/2020,$292.65,49678430,$286.53,$297.88,$286.5
3,02/25/2020,$288.08,57668360,$300.95,$302.53,$286.13
4,02/24/2020,$298.18,55548830,$297.26,$304.18,$289.23


In [14]:
df.columns

Index(['Date', ' Close', ' Volume', ' Open', ' High', ' Low'], dtype='object')

In [15]:
df1=df[' Close']

In [22]:
df1=df1.str.replace('$','')

In [23]:
df1.head()

0     273.36
1     273.52
2     292.65
3     288.08
4     298.18
Name:  Close, dtype: object

In [25]:
df1=df1.apply(float)

In [29]:
scaler=MinMaxScaler(feature_range= (0,1))
#scaler.fit(df1)
dfs=scaler.fit_transform(np.array(df1).reshape(-1,1))

In [31]:
dfs

array([[8.18942624e-01],
       [8.19480684e-01],
       [8.83812549e-01],
       ...,
       [2.30693463e-04],
       [0.00000000e+00],
       [6.72575693e-05]])

In [32]:
dfs.shape

(2518, 1)

In [56]:
train=dfs[:int(0.80*len(dfs)),:]
test=dfs[int(0.80*len(dfs)):,:]

def train_test(dataset,steps):
    trainX,trainY=[],[]
    for i in range(len(dataset)-steps-1):
        k=dfs[i:i+steps,0]
        trainX.append(k)
        trainY.append(dfs[i+steps,0])
    return trainX,trainY

x_train, y_train=train_test(train,200)
x_test, y_test=train_test(test,200)

x_train=np.array(x_train)
x_test=np.array(x_test)
y_train=np.array(y_train)
y_test=np.array(y_test)

In [61]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)

(1813, 200, 1)
(1813, 1)
(303, 200, 1)


In [58]:
x_train=x_train.reshape(x_train.shape[0],x_train.shape[1],1)
x_test=x_test.reshape(x_test.shape[0],x_test.shape[1],1)

(1813, 1)


In [63]:

model=tf.keras.Sequential([
    tf.keras.layers.LSTM(50, return_sequences=True,input_shape=(200,1) ),
    tf.keras.layers.LSTM(50, return_sequences=True ),
    tf.keras.layers.LSTM(50 ),
    tf.keras.layers.Dense(1)
])
model.compile(optimizer='adam',loss='mean_squared_error')
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10)

Epoch 1/10
57/57 [==============================] - 30s 524ms/step - loss: 0.0219 - val_loss: 0.0330
Epoch 2/10
57/57 [==============================] - 17s 302ms/step - loss: 0.0109 - val_loss: 0.0326
Epoch 3/10
57/57 [==============================] - 12s 217ms/step - loss: 0.0113 - val_loss: 0.0325
Epoch 4/10
57/57 [==============================] - 12s 209ms/step - loss: 0.0108 - val_loss: 0.0329
Epoch 5/10
57/57 [==============================] - 12s 207ms/step - loss: 0.0109 - val_loss: 0.0330
Epoch 6/10
57/57 [==============================] - 12s 205ms/step - loss: 0.0101 - val_loss: 0.0340
Epoch 7/10
57/57 [==============================] - 12s 203ms/step - loss: 0.0111 - val_loss: 0.0288
Epoch 8/10
57/57 [==============================] - 12s 209ms/step - loss: 0.0101 - val_loss: 0.0286
Epoch 9/10
57/57 [==============================] - 12s 208ms/step - loss: 0.0098 - val_loss: 0.0299
Epoch 10/10
57/57 [==============================] - 12s 207ms/step - loss: 0.0095 - val_lo

In [64]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 200, 50)           10400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 200, 50)           20200     
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 50,851
Trainable params: 50,851
Non-trainable params: 0
_________________________________________________________________
